# Master Calibration
Below are the libraries you will need for this script.

In [8]:
import pandas as pd
import numpy as np
import win32com.client as MyWinCOM
import win32com.client
from datetime import datetime
import os

# Path, Directory, and Loading File
When loading the file we use "dtype = str" to load everything as a object. You will receive an error during the excel script because python and excel treat date fields differently.

In [9]:
filepath = 'C:/Users/adhillon/Desktop/Test E-Comp/'
os.chdir(filepath)
file = 'FY20 Q3 Master Calibration File Test.xlsx'
df = pd.read_excel(file, sheet_name = "PERF DATA", dtype = str, skiprows = [0])

# Pre-Precessing
We want to remove all the nan's and exclude all empty Org 2 Managers. When the Org 2 Manager is blank it means the worker is an Org 2 Manager

In [10]:
df = df.fillna('')
df = df.replace(to_replace= 'nan', value = '')
df = df[df["Org 2"]!=""]

# Excel API
Win32com helps you to manipulate excel through an API. There are other libraries like openpyxl that are also able to manipulate excel; however, the drawback is openpyxl will delete exisiting charts. 

- First we dispatch the API
- Second we open the workbook

In [11]:
xl = win32com.client.Dispatch("Excel.Application")
wb = xl.Workbooks.Open("C:/Users/adhillon/Desktop/Test E-Comp/FY20 Q3 Master Calibration Org 2 Template.xlsm") # template

# Looping through Org 2
We create a big loop to go through all the Org2s

## Macro
The macro is used mainly for copy formulas, replacing N/As and formatting columns
- Sheets("PERF DATA").Cells.Replace What:="#N/A", Replacement:="", LookAt:=xlPart, SearchOrder:=xlByRows (Replaces #NA)
- lstrow1 = Sheets("PERF DATA").Cells(Rows.Count, 1).End(xlUp).Row (this function tell you the last row in the first column)
- Sheets("PERF DATA").Range("O3:O" & lstrow1).Formula (This function lets you copy a formula to the last row)
- Sheets("PERF DATA").Range("J:J").NumberFormat (Function formats the column to short date)

In [12]:
for Org2_Name in df["Org 2"].unique():
    df_filter = df[df["Org 2"]==Org2_Name]
    df2 = np.ascontiguousarray(df_filter)
    lta_df3 = df2.tolist()
    ws = wb.Worksheets("PERF DATA")
# Formulas and formatting
    ws.Range("A3:AO4462").ClearContents() #clear content
    start_row = 3
    start_col = 1
    ws.Range(ws.Cells(start_row,start_col),
        ws.Cells(start_row+len(df.index)-1,start_col+len(df.columns))).Value = lta_df3
    xl.Application.Run("ReplaceNA") # run macro
    # time.sleep(5)
# Summary Sheet Starts
    ws = wb.Worksheets("SUMMARY")
    row_manager = 28
    for x in sorted(df_filter['Org 3'].unique()):
            ws.Range(str("M"+str(row_manager))).Value= x
            row_manager+=2
    row_sub_bus = 69
    for x in sorted(df_filter['Sub Business Area'].unique()):
            ws.Range(str("M"+str(row_sub_bus))).Value= x
            row_sub_bus+=2
    row = 90
    for x in sorted(df_filter['Career Level '].unique()):
            ws.Range(str("M"+str(row))).Value= x
            row+=2
    save_path = "C:\\Users\\adhillon\\Desktop\\Test E-Comp\\Org 2 Master Calibration\\"
    filename = save_path+Org2_Name+" FY20 Q3 Calibration File.xlsm"
    wb.SaveAs(filename)
# Clean
    row_manager = 28
    for x in df_filter['Org 3'].unique():
            ws.Range(str("M"+str(row_manager))).Value= ""
            row_manager+=2
    row_sub_bus = 69
    for x in sorted(df_filter['Sub Business Area'].unique()):
            ws.Range(str("M"+str(row_sub_bus))).Value= ""
            row_sub_bus+=2
    row = 90
    for x in sorted(df_filter['Career Level '].unique()):
            ws.Range(str("M"+str(row))).Value= ""
            row+=2
    ws = None

# Quit Excel and Empty Variables
We do this to make sure there is no memory being misused. Also do not save the last file

In [13]:
wb.Close()
xl.Quit()
wb = None
sheet = None
book = None
excel = None